In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

from textblob import TextBlob
import nltk

In [3]:
file_path = '/home/daniele/x_twitter_engagement/data/dataset_twitter-scraper_2024-03-15_14-22-12-784.csv'
df = pd.read_csv(file_path)

df.head()

,conversation_id,created_at,favorite_count,full_text,id,is_pinned,is_quote_tweet,is_retweet,is_truncated,media/0/media_url,...,user/profile_image_url_https,user/screen_name,user/statuses_count,user/translator_type,user/verified,user_mentions/0/id_str,user_mentions/0/name,user_mentions/0/profile,user_mentions/0/screen_name,view_count
0,1519480761749016577,2022-04-28T00:56:58.000Z,4519149,Next I’m buying Coca-Cola to put the cocaine b...,1519480761749016577,False,False,False,False,NaN,...,https://pbs.twimg.com/profile_images/168332538...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN
1,1518623997054918657,2022-04-25T16:12:30.000Z,3054758,I hope that even my worst critics remain on Tw...,1518623997054918657,False,False,False,False,NaN,...,https://pbs.twimg.com/profile_images/168332538...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN
2,1519495072802390016,2022-04-28T01:53:50.000Z,2499261,Let’s make Twitter maximum fun!,1519495072802390016,False,False,False,False,NaN,...,https://pbs.twimg.com/profile_images/168332538...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN
3,1518677066325053441,2022-04-25T19:43:22.000Z,2466722,🚀💫♥️ Yesss!!! ♥️💫🚀 https://t.co/0T9HzUHuh6,1518677066325053441,False,False,False,False,https://pbs.twimg.com/media/FRNsuSFWUAUW6aP.jpg,...,https://pbs.twimg.com/profile_images/168332538...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN
4,1519495982723084290,2022-04-28T01:57:27.000Z,2429989,"Listen, I can’t do miracles ok https://t.co/z7...",1519495982723084290,False,False,False,False,https://pbs.twimg.com/media/FRZViwWX0AMsqQ1.jpg,...,https://pbs.twimg.com/profile_images/168332538...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN


In [4]:
# Creating a function to clean the tweets

def cleantwt (twt):
  emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

  twt = re.sub('RT', '', twt) # remove 'RT' from tweets
  twt = re.sub('#[A-Za-z0-9]+', '', twt) # remove the '#' from the tweets
  twt = re.sub('\\n', '', twt) # remove the '\n' character
  twt = re.sub('https?:\/\/\S+', '', twt) # remove the hyperlinks
  twt = re.sub('@[\S]*', '', twt) # remove @mentions
  twt = re.sub('^[\s]+|[\s]+$', '', twt) # remove leading and trailing whitespaces
  twt = re.sub(emoj, '', twt) # remove emojis
  return twt

In [5]:
# Create a new column with the cleaned tweets

df['cleaned_text'] = df['full_text'].apply(cleantwt)

In [6]:
df.drop(df[df['cleaned_text'] == ''].index, inplace = True)
df.duplicated().sum()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 0 to 98
Data columns (total 85 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   conversation_id                            70 non-null     int64  
 1   created_at                                 70 non-null     object 
 2   favorite_count                             70 non-null     int64  
 3   full_text                                  70 non-null     object 
 4   id                                         70 non-null     int64  
 5   is_pinned                                  70 non-null     bool   
 6   is_quote_tweet                             70 non-null     bool   
 7   is_retweet                                 70 non-null     bool   
 8   is_truncated                               70 non-null     bool   
 9   media/0/media_url                          26 non-null     object 
 10  media/0/type                     

In [7]:
df.head()

,conversation_id,created_at,favorite_count,full_text,id,is_pinned,is_quote_tweet,is_retweet,is_truncated,media/0/media_url,...,user/screen_name,user/statuses_count,user/translator_type,user/verified,user_mentions/0/id_str,user_mentions/0/name,user_mentions/0/profile,user_mentions/0/screen_name,view_count,cleaned_text
0,1519480761749016577,2022-04-28T00:56:58.000Z,4519149,Next I’m buying Coca-Cola to put the cocaine b...,1519480761749016577,False,False,False,False,NaN,...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN,Next I’m buying Coca-Cola to put the cocaine b...
1,1518623997054918657,2022-04-25T16:12:30.000Z,3054758,I hope that even my worst critics remain on Tw...,1518623997054918657,False,False,False,False,NaN,...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN,I hope that even my worst critics remain on Tw...
2,1519495072802390016,2022-04-28T01:53:50.000Z,2499261,Let’s make Twitter maximum fun!,1519495072802390016,False,False,False,False,NaN,...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN,Let’s make Twitter maximum fun!
3,1518677066325053441,2022-04-25T19:43:22.000Z,2466722,🚀💫♥️ Yesss!!! ♥️💫🚀 https://t.co/0T9HzUHuh6,1518677066325053441,False,False,False,False,https://pbs.twimg.com/media/FRNsuSFWUAUW6aP.jpg,...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN,Yesss!!!
4,1519495982723084290,2022-04-28T01:57:27.000Z,2429989,"Listen, I can’t do miracles ok https://t.co/z7...",1519495982723084290,False,False,False,False,https://pbs.twimg.com/media/FRZViwWX0AMsqQ1.jpg,...,elonmusk,40099,none,True,NaN,NaN,NaN,NaN,NaN,"Listen, I can’t do miracles ok"


In [26]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity
